In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

2023-01-26 05:02:03.828778: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.11.0'

In [3]:
!wget "https://unsplash.com/photos/dR_q93lfaTw/download?ixid=MnwxMjA3fDB8MXxhbGx8fHx8fHx8fHwxNjc0NjQ1Njc3&force=true&w=640" -O mountain.jpg

--2023-01-26 05:05:23--  https://unsplash.com/photos/dR_q93lfaTw/download?ixid=MnwxMjA3fDB8MXxhbGx8fHx8fHx8fHwxNjc0NjQ1Njc3&force=true&w=640
Resolving unsplash.com (unsplash.com)... 151.101.65.181, 151.101.129.181, 151.101.193.181, ...
Connecting to unsplash.com (unsplash.com)|151.101.65.181|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://images.unsplash.com/photo-1486870591958-9b9d0d1dda99?ixlib=rb-4.0.3&dl=jeremy-bishop-dR_q93lfaTw-unsplash.jpg&w=640&q=80&fm=jpg&crop=entropy&cs=tinysrgb [following]
--2023-01-26 05:05:24--  https://images.unsplash.com/photo-1486870591958-9b9d0d1dda99?ixlib=rb-4.0.3&dl=jeremy-bishop-dR_q93lfaTw-unsplash.jpg&w=640&q=80&fm=jpg&crop=entropy&cs=tinysrgb
Resolving images.unsplash.com (images.unsplash.com)... 2a04:4e42:48::720, 199.232.46.208
Connecting to images.unsplash.com (images.unsplash.com)|2a04:4e42:48::720|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66191 (65K) [image/jpeg]
Saving 

In [4]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet import preprocess_input

In [5]:
model = keras.models.load_model('mobilenet_v1_03_1.000.h5')

In [6]:
img = load_img('mountain.jpg', target_size=(224, 224))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [7]:
preds = model.predict(X)

1/1 [==============================] - 1s 572ms/step


In [8]:
preds[0]

array([-1.1973168, -7.126512 ,  3.2565868], dtype=float32)

In [9]:
classes = ['coast', 'forest', 'mountain']

In [10]:
dict(zip(classes, preds[0]))

{'coast': -1.1973168, 'forest': -7.126512, 'mountain': 3.2565868}

### Convert the model to TF-Lite 

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('scene-classifier-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/q6/4vk5yrts387cp_3qs430rhqc0000gn/T/tmpy8fza4rb/assets


INFO:tensorflow:Assets written to: /var/folders/q6/4vk5yrts387cp_3qs430rhqc0000gn/T/tmpy8fza4rb/assets
2023-01-26 05:13:07.400354: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-26 05:13:07.400384: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


## Using the TF-Lite Model

In [13]:
import tensorflow.lite as tflite

In [14]:
interpreter = tflite.Interpreter(model_path='scene-classifier-model.tflite')
interpreter.allocate_tensors()

In [17]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [18]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [20]:
classes = ['coast', 'forest', 'mountain']
dict(zip(classes, preds[0]))

{'coast': -1.1973078, 'forest': -7.126517, 'mountain': 3.2565804}

We've verified that the result from the TF-Lite model is the same as the original model.